In [1]:
# Install LangChain, LangGraph, Gemini SDK
!pip install -q langchain langgraph google-generativeai langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.5 MB/s eta 0:00:00


In [2]:
#Import all dependencies
import os
import getpass
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

In [3]:
# Secure input for Gemini API key
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Gemini API key: ")

Enter your Gemini API key: ··········


In [4]:
# Use Gemini Pro with LangChain wrapper
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0.7)

In [5]:
# Evaluate burnout based on user inputs
def evaluate_burnout_risk(mood: int, sleep: float, breaks: float) -> str:
    risk_flags = 0
    if mood <= 3:
        risk_flags += 1
    if sleep < 6:
        risk_flags += 1
    if breaks < 1:
        risk_flags += 1
    return "high" if risk_flags >= 2 else "low"

In [6]:
from typing import TypedDict
# Define the state schema
class State(TypedDict):
    mood: int
    sleep_hours: float
    break_hours: float
    risk_level: str
    recommendations: str
    affirmation: str

# Start Node
def start_node(state: State) -> State:
    print("👋 Hello! I'm your self-care AI assistant.")
    print("Let's check how you're doing today.\n")

    # Simulated inputs (replace with actual input for interactive use)
    mood = int(input("How would you rate your mood today (1 = bad, 5 = great)? "))
    sleep_hours = float(input("How many hours did you sleep last night? "))
    break_hours = float(input("How many hours did you take proper breaks today? "))

    return {"mood": mood, "sleep_hours": sleep_hours, "break_hours": break_hours}

In [7]:
# Analyzer node
def analyzer_node(state: State) -> State:
    mood = state.get("mood", 3)
    sleep = state.get("sleep_hours", 6.0)
    breaks = state.get("break_hours", 1.0)

    risk = evaluate_burnout_risk(mood, sleep, breaks)
    print(f"📊 Burnout Risk Level: {risk.upper()}")

    return {"risk_level": risk}

In [8]:
# Uses Gemini to suggest 3 simple self-care activities
def recommendation_node(state: State) -> State:
    mood = state["mood"]
    sleep = state["sleep_hours"]
    breaks = state["break_hours"]

    prompt = ChatPromptTemplate.from_messages([
        ("system", "You're a self-care advisor for university students."),
        ("human", f"""Suggest 3 simple, practical self-care activities for a student who is showing signs of burnout:
              Mood rating: {mood}
              Sleep hours: {sleep}
              Breaks today: {breaks}
              Be gentle, specific, and encouraging.""")
    ])

    chain = prompt | llm
    response = chain.invoke({})
    suggestion = response.content.strip()

    print("\n🧘 Self-Care Recommendations:\n")
    print(suggestion)
    return state
    #return {"recommendations": suggestion}


In [9]:
def affirmation_node(state: State) -> State:
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You're a motivational self-care coach."),
        ("human", "The student is doing well and not at burnout risk. Write a short, encouraging affirmation.")
    ])
    chain = prompt | llm
    response = chain.invoke({})
    affirmation = response.content.strip()

    print("\n🌟 Affirmation:\n")
    print(affirmation)

    return state
    #return {"affirmation": affirmation}

In [10]:
def router_node(state: State) -> str:
    return "recommendation" if state["risk_level"] == "high" else "affirmation"

In [11]:
# Define the LangGraph flow using StateGraph
builder = StateGraph(State)

# Add nodes
builder.add_node("start", start_node)
builder.add_node("analyzer", analyzer_node)
#builder.add_node("router", router_node)
builder.add_node("recommendation", recommendation_node)
builder.add_node("affirmation", affirmation_node)

# Define edges
builder.set_entry_point("start")
builder.add_edge("start", "analyzer")
#builder.add_edge("analyzer", "router")
builder.add_conditional_edges("analyzer", router_node, {
    "recommendation": "recommendation",
    "affirmation": "affirmation"
})


builder.add_edge("recommendation", END)
builder.add_edge("affirmation", END)

# Compile graph
app = builder.compile()


In [12]:
# Invoke the graph
final_state = app.invoke({})

👋 Hello! I'm your self-care AI assistant.
Let's check how you're doing today.

How would you rate your mood today (1 = bad, 5 = great)? 4
How many hours did you sleep last night? 7
How many hours did you take proper breaks today? 2
📊 Burnout Risk Level: LOW

🌟 Affirmation:

You're thriving!  Keep shining brightly and nurturing your amazing progress.  You've got this!
